In [823]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv

In [824]:
## Configuration 

In [825]:
#etl = 'kobo2elastic'
#etl = 'curis2elastic'
#etl = 'oldcuris2newcuris'
etl = 'isabela2newaqm'
#etl = 'cambodia2newaqm'

input_schema_file = ''
input_data_file = ''
mapping_file = ''

root_object = 'resident'

if etl == 'curis2elastic':
    #old curis to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.1-items.json'
    mapping_file = 'schema/map/couchbase2elastic.map.csv'
    output_dir = 'file/curisSchema/'
    
elif etl == 'kobo2elastic':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/koboSchema.1-item.json'
    input_data_file = 'data/koboData.2-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/koboSchema/'
    
elif etl == 'oldcuris2newcuris':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/curisData.1-item.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/curisSchema/'
    
elif etl == 'cambodia2newaqm':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/KHMSchema.json'
    input_data_file = 'data/KHM.3-items.json'
    mapping_file = 'schema/map/kobo2elastic.map.csv'
    output_dir = 'file/KHMSchema/'

elif etl == 'isabela2newaqm':
    #kobo to elasticsearch
    input_schema_file = 'schema/input/curisSchema.1-item.json'
    input_data_file = 'data/Isabela.2-items.json'
    mapping_file = 'schema/map/Isabela2newAQM.map.csv'
    #output_dir = 'file/IsabelaSchema/'
    output_dir = 'data/processed/couchbase-curis-2019-06-21/'

In [826]:
def _open_file():
    with open(input_data_file) as f:
        return json.load(f)

In [827]:
def _flatten_json(nested_json):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '.')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + '' + str(i) + '.')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [828]:
def _dict_to_dataframe(dict_object):
    return pd.DataFrame.from_dict({'value': dict_object})

In [829]:
## OPTIMIZE TO ACCEPT LIST NOT DATAFRAME
def filter_key(x):
    lists = format_key(x)
    lists = exclude_digit(lists)
    lists = list2string(lists)
    return lists

def filter_index_map(x):
    lists = format_key(x)
    lists = include_digit(lists)
    lists = list2string(lists)
    return lists

def format_key(items):
    #return list(map(lambda x:x.lower().split(sep='.'), items ))
    return items.lower().split(sep='.')

def include_digit(items):
    return [item for item in items if item.isdigit()]

def exclude_digit(items):
    return [item for item in items if not item.isdigit()]

def list2string(lists):
    return '.'.join(lists)

In [830]:
def _add_custom_colums(dataframe_object):
    dataframe_object['key'] = list(dataframe_object.index)
    dataframe_object['key'] = dataframe_object['key'].apply(filter_key)
    #print('_add_custom_columns ',dataframe_object[dataframe_object['key'] == 'id' ])

    
    #TODO: musbe adjusted dynamically
    
    #CHECK if single object or array of objects
    #formid == koboschema
    #id == curischema
    #dataframe_object['_id'] = dataframe_object[dataframe_object['key'] == 'formid' ]['value'].values[0]
    
    #IF oldcursi2newcuris
    dataframe_object['_id'] = dataframe_object[dataframe_object['key'] == 'id' ]['value'].values[0]
    #IF KHMSChema
    #dataframe_object['_id'] = dataframe_object[dataframe_object['key'] == '_id' ]['value'].values[0]
    
    dataframe_object['_index_map'] = list(dataframe_object.index)
    dataframe_object['_index_map'] = dataframe_object['_index_map'].apply(filter_index_map)
    #print(dataframe_object)
    dataframe_object = dataframe_object.reset_index(drop=True)

    return dataframe_object

In [831]:
## File loader

In [832]:
def _get_mapping_fields():
    _mapping_df = pd.DataFrame()
    _mapping_df = read_csv(mapping_file).sort_values(['source_key'])

    _required_mapping_fields = list(_mapping_df['source_key'])
    return _required_mapping_fields

In [833]:
def _get_index_map(data_df):
    return list(data_df['_index_map'].unique())

In [834]:
def _filter_by_indexmap(data_df, index):
    return data_df[data_df['_index_map'] == index ]

In [835]:
def clean_value(x):
    return x.lower().replace("/", ".").replace("the", "").replace("schema", "").strip()

In [836]:
def _get_filenames():
    schema_csv = pd.read_csv(output_dir + 'schema.csv', skiprows=0)
    return schema_csv['file_name'].values[0].split(sep=",")

In [837]:
def _get_csv_headers(filename):
    return pd.read_csv( output_dir + filename + '.csv',nrows=0) #get header only

In [838]:
def _get_required_data(input_data_df,csv_header_df):
    
    required_header_list = [] 

    for data in input_data_df['key']:
        
        if data in csv_header_df.columns:
            
            required_header_list.append(data)
    
    #print(required_header_list)
    
    return input_data_df[input_data_df['key'].isin(required_header_list)]

In [839]:
def _rename_index(data_df):
    if 'key' in data_df.columns:
        data_df.index = list(data_df['key'])
        data_df = data_df.drop('key', axis=1)
    return data_df 

In [840]:
def _columnar_to_row(input_data_df,csv_header_df):
    
    for header in list(input_data_df.index):
        if header:
            csv_header_df.at['',header] = input_data_df.loc[header]['value'] 
    

        csv_header_df.at['','_id'] = input_data_df['_id'][header] 
        csv_header_df.at['','_index_map'] = input_data_df['_index_map'][header] 
    return csv_header_df

In [841]:
def _write_to_csv(data_csv_df, filename):
    return data_csv_df.to_csv(output_dir + filename + '.csv', encoding='utf-8', mode='a', header=False,index=False)

## OPEN DATA FROM FILE

In [842]:
data_list = []
data_list = _open_file()
data_list

[{'_sync': {'history': {'channels': [None, None, None, None, None, None],
    'parents': [5, -1, 1, 2, 3, 4],
    'revs': ['6-de6706bf00721558205d0f5651901f94',
     '1-e86d3945fadafa4f84f5e478841809d1',
     '2-c4096a64f5e91372cc97daeded4955dc',
     '3-a03d7327410cd5b1e0638a8f9b8bd388',
     '4-4df5f344f4a781868812806b76e50a87',
     '5-218399a922e1f7ffbe64107b4508f5a0']},
   'recent_sequences': [33481],
   'rev': '6-de6706bf00721558205d0f5651901f94',
   'sequence': 33481,
   'time_saved': '2018-08-21T07:42:41.523087917Z'},
  'address': {'barangay': 'Bangad',
   'country': 'Philippines',
   'lot_or_house_number': '',
   'postal_code': '3330',
   'province': 'Isabela'},
  'birthdate': '08/09/1989',
  'contact_number': {'country_code': '+63', 'number': None},
  'profiles': [{'civil_status': 'Single',
    'date_updated': '07/02/2018 at 08:44:13 PM GMT+08:00',
    'education': 'Masters Level',
    'employment': {'is_employed': True,
     'monthly_income': 'Php12.00',
     'nature': 'empt

## FLATTEN JSON FILE -- ITERATE DATA HERE

In [944]:
datum = data_list[1]
test_data_flat_dict = _flatten_json(datum)  
test_data_flat_dict

{'_sync.history.channels.0': None,
 '_sync.history.channels.1': None,
 '_sync.history.channels.2': None,
 '_sync.history.channels.3': None,
 '_sync.history.channels.4': None,
 '_sync.history.channels.5': None,
 '_sync.history.channels.6': None,
 '_sync.history.channels.7': None,
 '_sync.history.channels.8': None,
 '_sync.history.channels.9': None,
 '_sync.history.channels.10': None,
 '_sync.history.channels.11': None,
 '_sync.history.channels.12': None,
 '_sync.history.channels.13': None,
 '_sync.history.parents.0': 10,
 '_sync.history.parents.1': 11,
 '_sync.history.parents.2': 5,
 '_sync.history.parents.3': 0,
 '_sync.history.parents.4': 7,
 '_sync.history.parents.5': 1,
 '_sync.history.parents.6': 12,
 '_sync.history.parents.7': -1,
 '_sync.history.parents.8': 4,
 '_sync.history.parents.9': 8,
 '_sync.history.parents.10': 9,
 '_sync.history.parents.11': 3,
 '_sync.history.parents.12': 2,
 '_sync.history.parents.13': 6,
 '_sync.history.revs.0': '6-8c1d02c2e0e74568dabbc8eb0395585e',
 

## Dictionary TO DATAFRAME

In [945]:
test_data_flat_df = _dict_to_dataframe(test_data_flat_dict)
test_data_flat_df

,value
_sync.history.channels.0,None
_sync.history.channels.1,None
_sync.history.channels.10,None
_sync.history.channels.11,None
_sync.history.channels.12,None
_sync.history.channels.13,None
_sync.history.channels.2,None
_sync.history.channels.3,None
_sync.history.channels.4,None
_sync.history.channels.5,None


## ADD CUSTOM COLUMNS WITHOUT MAPPING

In [946]:
_customed_data_flat_df = _add_custom_colums(test_data_flat_df)
_customed_data_flat_df

,value,key,_id,_index_map
0,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,0
1,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,1
2,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,10
3,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,11
4,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,12
5,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,13
6,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,2
7,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,3
8,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,4
9,None,_sync.history.channels,2f920d39-319e-4bf7-90c8-6133c3337af7,5


## GET REQUIRED MAPPING

In [947]:
_required_mapping_fields = []
_required_mapping_fields = _get_mapping_fields()
_required_mapping_fields

['address.province',
 'birthdate',
 'contact_number.number',
 'email_address',
 'family_members',
 'first_name',
 'gender',
 'households.house_ownership',
 'households.neighborhood_description',
 'households.sanitary_ownership',
 'id',
 'last_name',
 'middle_name',
 'profiles.civil_status',
 'registered_at',
 'user-cam.id',
 nan]

## FILTER ONLY REQUIRED MAPPING FROM RAW FLAT DATA

In [948]:
_customed_data_flat_df = _customed_data_flat_df[_customed_data_flat_df['key'].isin(_required_mapping_fields)]
_customed_data_flat_df

,value,key,_id,_index_map
50,Isabela,address.province,2f920d39-319e-4bf7-90c8-6133c3337af7,
51,12/23/1975,birthdate,2f920d39-319e-4bf7-90c8-6133c3337af7,
53,None,contact_number.number,2f920d39-319e-4bf7-90c8-6133c3337af7,
54,None,email_address,2f920d39-319e-4bf7-90c8-6133c3337af7,
55,2f920d39-319e-4bf7-90c8-6133c3337af7,family_members,2f920d39-319e-4bf7-90c8-6133c3337af7,0
56,,first_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
57,Male,gender,2f920d39-319e-4bf7-90c8-6133c3337af7,
58,Owned Not,households.house_ownership,2f920d39-319e-4bf7-90c8-6133c3337af7,0
59,Urban,households.neighborhood_description,2f920d39-319e-4bf7-90c8-6133c3337af7,0
62,Owned Not,households.sanitary_ownership,2f920d39-319e-4bf7-90c8-6133c3337af7,0


## GET HEADERS THRU META DATA

In [943]:
test_meta_header_df = pd.read_csv( 'data/processed/couchbase-curis-2019-06-21/_meta.csv')
test_meta_header_df

,file_name,field_name,field_type
0,households,households.type_of_house,string
1,households,households.house_ownership,string
2,households,households.sanitary_ownership,string
3,households,households.neighborhood_description,string
4,profiles,profiles.civil_status,string
5,resident,middle_name,string
6,resident,birthdate,string
7,resident,id,string
8,resident,type,string
9,resident,first_name,string


## GET UNIQUE FILENAME -- ITERATION

In [960]:
_filename_list = list(test_meta_header_df['file_name'].unique())
_filename_list

['households', 'profiles', 'resident']

In [961]:
_filename = _filename_list[2]
_filename

'resident'

## GET ALL UNIQUE INDEX MAP -- ITERATION

In [897]:
test_index_map =  list(_customed_data_flat_df['_index_map'].unique())
test_index_map

['', '0']

In [963]:
_test_index_map = test_index_map[0]
_test_index_map 

''

## FILTER ONLY REQUIRED  DATA ACCORDING TO INDEX MAP

In [964]:
test_source_data_df = pd.DataFrame()
test_source_data_df = _filter_by_indexmap(_customed_data_flat_df, _test_index_map)
test_source_data_df

,value,key,_id,_index_map
50,Isabela,address.province,2f920d39-319e-4bf7-90c8-6133c3337af7,
51,12/23/1975,birthdate,2f920d39-319e-4bf7-90c8-6133c3337af7,
53,None,contact_number.number,2f920d39-319e-4bf7-90c8-6133c3337af7,
54,None,email_address,2f920d39-319e-4bf7-90c8-6133c3337af7,
56,,first_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
57,Male,gender,2f920d39-319e-4bf7-90c8-6133c3337af7,
65,2f920d39-319e-4bf7-90c8-6133c3337af7,id,2f920d39-319e-4bf7-90c8-6133c3337af7,
72,"""Raphy Canceran TO DELETE""",last_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
74,Laddran,middle_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
86,07/12/2018 at 10:54:14 PM GMT+08:00,registered_at,2f920d39-319e-4bf7-90c8-6133c3337af7,


## CLEAN FILTERED BY INDEX_MAP #Cleaning is for '/' in cambodia schema

In [965]:
test_source_data_df['key'].apply(clean_value)
test_source_data_df

,value,key,_id,_index_map
50,Isabela,address.province,2f920d39-319e-4bf7-90c8-6133c3337af7,
51,12/23/1975,birthdate,2f920d39-319e-4bf7-90c8-6133c3337af7,
53,None,contact_number.number,2f920d39-319e-4bf7-90c8-6133c3337af7,
54,None,email_address,2f920d39-319e-4bf7-90c8-6133c3337af7,
56,,first_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
57,Male,gender,2f920d39-319e-4bf7-90c8-6133c3337af7,
65,2f920d39-319e-4bf7-90c8-6133c3337af7,id,2f920d39-319e-4bf7-90c8-6133c3337af7,
72,"""Raphy Canceran TO DELETE""",last_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
74,Laddran,middle_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
86,07/12/2018 at 10:54:14 PM GMT+08:00,registered_at,2f920d39-319e-4bf7-90c8-6133c3337af7,


## GET FIELDS FOR EACH FILENAME -- ITERATION OF FILENAME

In [952]:
test_meta_header_df[test_meta_header_df['file_name'] == _filename ]

,file_name,field_name,field_type
5,resident,middle_name,string
6,resident,birthdate,string
7,resident,id,string
8,resident,type,string
9,resident,first_name,string
10,resident,last_name,string
11,resident,last_name_suffix,string
12,resident,organization,string
13,resident,nhid,string
14,resident,gender,string


In [966]:
test_list_field_name = list(test_meta_header_df[test_meta_header_df['file_name'] == _filename ]['field_name'])
test_list_field_name 

['middle_name',
 'birthdate',
 'id',
 'type',
 'first_name',
 'last_name',
 'last_name_suffix',
 'organization',
 'nhid',
 'gender',
 'registered_at',
 'address.postal_code',
 'address.country',
 'address.barangay',
 'identification.id3.type',
 'address.lot_or_house_number',
 'address.province',
 'identification.id1.type',
 'identification.id2.type',
 'profile_picture.path',
 'contact_number.number',
 'user-cam.id',
 'contact_number.country_code',
 'profile_picture.name',
 'identification.id2.identifier',
 'identification.id3.identifier',
 'identification.id1.identifier',
 'user-cam.owner']

## *NEW* GET REQUIRED DATA ALGO -- NO ITERATION

In [967]:
test_required_header_list = [] 
test_required_header_list = set(test_source_data_df['key']) & set(test_list_field_name)
test_source_data_df = test_source_data_df[test_source_data_df['key'].isin(test_required_header_list)]
test_source_data_df

,value,key,_id,_index_map
50,Isabela,address.province,2f920d39-319e-4bf7-90c8-6133c3337af7,
51,12/23/1975,birthdate,2f920d39-319e-4bf7-90c8-6133c3337af7,
53,None,contact_number.number,2f920d39-319e-4bf7-90c8-6133c3337af7,
56,,first_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
57,Male,gender,2f920d39-319e-4bf7-90c8-6133c3337af7,
65,2f920d39-319e-4bf7-90c8-6133c3337af7,id,2f920d39-319e-4bf7-90c8-6133c3337af7,
72,"""Raphy Canceran TO DELETE""",last_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
74,Laddran,middle_name,2f920d39-319e-4bf7-90c8-6133c3337af7,
86,07/12/2018 at 10:54:14 PM GMT+08:00,registered_at,2f920d39-319e-4bf7-90c8-6133c3337af7,
88,csoriano@gmail.com,user-cam.id,2f920d39-319e-4bf7-90c8-6133c3337af7,


## ===== WRITE DATA ======

## GET CSV HEADERS OF EACH FILENAME

In [968]:
test_csv_header_df = pd.DataFrame()
test_csv_header_df = _get_csv_headers(_filename)  #Resident,39 #Household,5
test_csv_header_df

,_id,_index_map,middle_name,birthdate,id,type,first_name,last_name,last_name_suffix,organization,...,identification.id2.type,profile_picture.path,contact_number.number,user-cam.id,contact_number.country_code,profile_picture.name,identification.id2.identifier,identification.id3.identifier,identification.id1.identifier,user-cam.owner


## GET REQUIRED DATA -- deprecated

In [935]:
#test_new_data_df = pd.DataFrame()
#test_new_data_df = _get_required_data(test_source_data_df, test_csv_header_df)
#test_new_data_df

## RENAME REQUIRED DATA

In [969]:
test_new_data_df = _rename_index(test_source_data_df)
test_new_data_df

,value,_id,_index_map
address.province,Isabela,2f920d39-319e-4bf7-90c8-6133c3337af7,
birthdate,12/23/1975,2f920d39-319e-4bf7-90c8-6133c3337af7,
contact_number.number,None,2f920d39-319e-4bf7-90c8-6133c3337af7,
first_name,,2f920d39-319e-4bf7-90c8-6133c3337af7,
gender,Male,2f920d39-319e-4bf7-90c8-6133c3337af7,
id,2f920d39-319e-4bf7-90c8-6133c3337af7,2f920d39-319e-4bf7-90c8-6133c3337af7,
last_name,"""Raphy Canceran TO DELETE""",2f920d39-319e-4bf7-90c8-6133c3337af7,
middle_name,Laddran,2f920d39-319e-4bf7-90c8-6133c3337af7,
registered_at,07/12/2018 at 10:54:14 PM GMT+08:00,2f920d39-319e-4bf7-90c8-6133c3337af7,
user-cam.id,csoriano@gmail.com,2f920d39-319e-4bf7-90c8-6133c3337af7,


## COLUMNAR TO ROW REQIURED DATA

In [970]:
test_new_data_df = _new_columnar_to_row(test_new_data_df, test_csv_header_df)
test_new_data_df

,_id,_index_map,middle_name,birthdate,id,type,first_name,last_name,last_name_suffix,organization,...,identification.id2.type,profile_picture.path,contact_number.number,user-cam.id,contact_number.country_code,profile_picture.name,identification.id2.identifier,identification.id3.identifier,identification.id1.identifier,user-cam.owner
,2f920d39-319e-4bf7-90c8-6133c3337af7,,Laddran,12/23/1975,2f920d39-319e-4bf7-90c8-6133c3337af7,NaN,,"""Raphy Canceran TO DELETE""",NaN,NaN,...,NaN,NaN,None,csoriano@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN


## WRITE TO CSV

In [938]:
test_new_data_df = _write_to_csv(test_new_data_df, _filename)
test_new_data_df 

## MAPPING FIELDS

In [863]:
## =================================================

In [864]:
## Get Index_map unique values

In [865]:
## BUG: IF ADDRESS IS 
'''
    "answers": {
        "Address": [{
            "Location": {
                "Street": ["12312", "234234"],
                "City": "City"
            }
        }],
'''

'\n    "answers": {\n        "Address": [{\n            "Location": {\n                "Street": ["12312", "234234"],\n                "City": "City"\n            }\n        }],\n'